In [1]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn import utils

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [2]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)
genre_set = {'Comedy'}

In [3]:
def text_to_list(x):
    if pd.isna(x):
        return ''
    else:
        return ast.literal_eval(x)

def parse_json(x):
    try:
        return json.loads(x.replace("'", '"'))[0]['name']
    except:
        return ''
    
def parse_all_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        for i in range(numElems):
            genre_set.add(json_genres[i]['name'])
    except:
        return ''
    
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #20 0s
        for i in range(numElems):
            ret[genre_dict[(json_genres[i]['name'])]] = 1
        return ret
    except:
        return ''
    

def get_labels_as_strs(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = []#20 0s
        for i in range(numElems):
            ret.append(json_genres[i]['name'])
        return ret
    except:
        return ''

In [4]:
 def getAllGenres():
    full_data = pd.read_csv('train.csv')

    y = full_data['genres']
    y.apply(parse_all_genres_json)

In [5]:
getAllGenres()#populate the genre set

In [6]:
#get set to dictionary for indexing of target vectors
genre_dict = {}
index = 0
for genre in genre_set:
    genre_dict[genre] = index
    index += 1

In [7]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret
    label_strs = y.apply(get_labels_as_strs)
    all_data['genres_labels'] = label_strs
    return ret

In [8]:
labels_vects = getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [9]:
#put to lower case, remove punctation
def cleanText(text):
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text
all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [10]:
#logistic regression data
lr_data = all_data[['cleanOverview', 'genres_labels', 'genres_vect', 'overview']]

In [11]:
train, test = train_test_split(lr_data, test_size=0.2, random_state=42)

CNN STUFF here

In [12]:
#get word embeddings
x = train['cleanOverview'].values.tolist()
y = train['genres_vect']

In [13]:
x_test = test['cleanOverview'].values.tolist()
y_test = test['genres_vect']

In [14]:
y_train = y.tolist()
y_train = np.array(y_train)

In [15]:
y_test = y_test.tolist()
y_test = np.array(y_test)

In [16]:
tok = [word_tokenize(sent) for sent in x]

In [17]:
word_vec_len = 32
model = Word2Vec(tok, min_count = 1, size=word_vec_len)

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words_kept = 100000 #using 100000 most popular words, use throughout

tokenizer = Tokenizer(num_words_kept)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

x_train_seq = pad_sequences(sequences, maxlen=200)

Using TensorFlow backend.


In [19]:
test_sequences = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(test_sequences, maxlen=200)

In [20]:
embeddings_index = {}
for w in model.wv.vocab.keys():
    embeddings_index[w] = model.wv[w]


embedding_matrix = np.zeros((num_words_kept, word_vec_len))
for word, i in tokenizer.word_index.items():
    if i >= num_words_kept:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def get_per_label_metrics(real_labels_matrix, predictions_labels_matrix):
    for genre in genre_dict.keys():
        index = genre_dict[genre]
        real_labels_vect = real_labels_matrix[:, index]
        prediction_vect = predictions_labels_matrix[:,index]
        print("Accuracy for " + genre + ": " + str(accuracy_score(real_labels_vect, prediction_vect)))
        print("Precision for " + genre + ": " + str(precision_score(real_labels_vect, prediction_vect)))
        print("Recall for " + genre + ": " + str(recall_score(real_labels_vect, prediction_vect)))
        print()

In [22]:
#size of intersection of predicted and actual labels divided by size of their union for each datapoint tested on
#sum those and then divide by number of datapoints
#vectorized for speed
def multi_label_accuracy(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    union = real_labels_matrix | predictions_labels_matrix
    #sum(array.T) gets number of 1s in row
    row_wise_accuracy = sum(intersection.T) / sum(union.T)
    return sum(row_wise_accuracy) / real_labels_matrix.shape[0]

#size of intersection of predicted and actual labels divided by size of predicted set for each datapoint tested on
#sum those and divide by number of datapoints
#if no predicted labels, don't count that row towards the precision as that would be undefined
def multi_label_precision(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    precision_sum = 0
    num_rows = 0
    for row in range(intersection.shape[0]):
        if sum(predictions_labels_matrix[row]) > 0: #if there is at least one prediction for this row
            num_rows += 1
            precision_sum += sum(intersection[row]) / sum(predictions_labels_matrix[row])
    if num_rows == 0:
        return 0#no labels predicted at all will give us 0 precision as precision makes no sense here
    return precision_sum / num_rows

#size of intersection of predicted and actual labels divided by size of real label set for each datapoint tested on
#sum those and divide by number of datapoints
#all datapoints should have at least 1 real label in this data set
#vectorized for speed
def multi_label_recall(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    #sum(array.T) gets number of 1s in row
    row_wise_recall = sum(intersection.T) / sum(real_labels_matrix.T)
    return sum(row_wise_recall) / real_labels_matrix.shape[0]

#lower is better
def hamming_loss(real_labels_matrix, predictions_labels_matrix):
    return (np.logical_xor(real_labels_matrix, predictions_labels_matrix)).sum()/(real_labels_matrix.shape[0] * real_labels_matrix.shape[1])

import keras.backend as K

#metric for keras for early stopping
#takes in raw labels from kerass (not yet converted to 0 and 1s)
#NOT the same as accuracy, this is total labels correctly identified divided by union of total labels
#this weights rows with more labels higher, where accruacy does not, but this is still a good metric for early stopping
def raw_multi_label_accuracy(y_true, y_pred):
    positives = K.greater_equal(y_pred, 0.5)
    positives = K.cast(positives, K.floatx())
    new_y_pred = positives #+ ((1-positives)*y_pred)
    intersection = y_true * new_y_pred
    union = 1 -((1-y_true)*(1-new_y_pred))
    accuracy = K.sum(intersection) / K.sum(union)
    return accuracy
    

In [23]:
from keras.callbacks import EarlyStopping
#for early stopping only after certain number of epochs. wait until delay epochs until early stopping
class DelayedEarlyStopping(EarlyStopping):
    def __init__(self, monitor, min_delta=0, patience=0, verbose=0, mode='auto', delay = 100):
        super(DelayedEarlyStopping, self).__init__()
        self.delay = delay

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.delay:
            super().on_epoch_end(epoch, logs)

In [24]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

model_cnn = Sequential()
#e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=200, trainable=False)
#e = Embedding(num_words_kept, word_vec_len, input_length=200, trainable=True)
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=200, trainable=True)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dense(20, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
#model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
model_cnn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 10, delay=250)], epochs=1000, batch_size=100, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 2149 samples, validate on 239 samples
Epoch 1/1000
 - 3s - loss: 0.4097 - raw_multi_label_accuracy: 0.1018 - val_loss: 0.3233 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 2s - loss: 0.3166 - raw_multi_label_accuracy: 0.0996 - val_loss: 0.3162 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 2s - loss: 0.3131 - raw_multi_label_accuracy: 0.0894 - val_loss: 0.3144 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 3s - loss: 0.3124 - raw_multi_label_accuracy: 0.1173 - val_loss: 0.3158 - val_raw_multi_label_accuracy: 0.1683
Epoch 5/1000
 - 3s - loss: 0.3115 - raw_multi_label_accuracy: 0.1132 - val_loss: 0.3151 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 6/1000
 - 3s - loss: 0.3100 - raw_multi_label_accuracy: 0.0928 - val_loss: 0.3152 - val_raw_multi_

Epoch 58/1000
 - 2s - loss: 9.2608e-04 - raw_multi_label_accuracy: 0.9994 - val_loss: 0.8952 - val_raw_multi_label_accuracy: 0.1627
Epoch 59/1000
 - 3s - loss: 8.7235e-04 - raw_multi_label_accuracy: 0.9995 - val_loss: 0.9011 - val_raw_multi_label_accuracy: 0.1632
Epoch 60/1000
 - 2s - loss: 8.2327e-04 - raw_multi_label_accuracy: 0.9994 - val_loss: 0.9086 - val_raw_multi_label_accuracy: 0.1586
Epoch 61/1000
 - 2s - loss: 7.8646e-04 - raw_multi_label_accuracy: 0.9994 - val_loss: 0.9142 - val_raw_multi_label_accuracy: 0.1594
Epoch 62/1000
 - 2s - loss: 7.4280e-04 - raw_multi_label_accuracy: 0.9994 - val_loss: 0.9162 - val_raw_multi_label_accuracy: 0.1606
Epoch 63/1000
 - 2s - loss: 7.0752e-04 - raw_multi_label_accuracy: 0.9995 - val_loss: 0.9241 - val_raw_multi_label_accuracy: 0.1580
Epoch 64/1000
 - 3s - loss: 6.7582e-04 - raw_multi_label_accuracy: 0.9994 - val_loss: 0.9268 - val_raw_multi_label_accuracy: 0.1594
Epoch 65/1000
 - 3s - loss: 6.4355e-04 - raw_multi_label_accuracy: 0.9994 - 

Epoch 120/1000
 - 2s - loss: 1.6547e-04 - raw_multi_label_accuracy: 0.9998 - val_loss: 1.0853 - val_raw_multi_label_accuracy: 0.1638
Epoch 121/1000
 - 2s - loss: 1.7449e-04 - raw_multi_label_accuracy: 0.9995 - val_loss: 1.0880 - val_raw_multi_label_accuracy: 0.1621
Epoch 122/1000
 - 3s - loss: 1.6822e-04 - raw_multi_label_accuracy: 0.9994 - val_loss: 1.0929 - val_raw_multi_label_accuracy: 0.1617
Epoch 123/1000
 - 5s - loss: 1.6634e-04 - raw_multi_label_accuracy: 0.9995 - val_loss: 1.0912 - val_raw_multi_label_accuracy: 0.1634
Epoch 124/1000
 - 2s - loss: 1.6385e-04 - raw_multi_label_accuracy: 0.9994 - val_loss: 1.0949 - val_raw_multi_label_accuracy: 0.1621
Epoch 125/1000
 - 4s - loss: 1.6313e-04 - raw_multi_label_accuracy: 0.9994 - val_loss: 1.0976 - val_raw_multi_label_accuracy: 0.1619
Epoch 126/1000
 - 5s - loss: 1.6193e-04 - raw_multi_label_accuracy: 0.9993 - val_loss: 1.0980 - val_raw_multi_label_accuracy: 0.1617
Epoch 127/1000
 - 3s - loss: 1.6003e-04 - raw_multi_label_accuracy: 0

Epoch 182/1000
 - 2s - loss: 1.1584e-04 - raw_multi_label_accuracy: 0.9995 - val_loss: 1.1741 - val_raw_multi_label_accuracy: 0.1609
Epoch 183/1000
 - 3s - loss: 1.2120e-04 - raw_multi_label_accuracy: 0.9990 - val_loss: 1.1721 - val_raw_multi_label_accuracy: 0.1640
Epoch 184/1000
 - 3s - loss: 1.1811e-04 - raw_multi_label_accuracy: 0.9993 - val_loss: 1.1771 - val_raw_multi_label_accuracy: 0.1591
Epoch 185/1000
 - 2s - loss: 1.2319e-04 - raw_multi_label_accuracy: 0.9989 - val_loss: 1.1726 - val_raw_multi_label_accuracy: 0.1637
Epoch 186/1000
 - 2s - loss: 1.1666e-04 - raw_multi_label_accuracy: 0.9991 - val_loss: 1.1791 - val_raw_multi_label_accuracy: 0.1622
Epoch 187/1000
 - 2s - loss: 1.1647e-04 - raw_multi_label_accuracy: 0.9992 - val_loss: 1.1759 - val_raw_multi_label_accuracy: 0.1630
Epoch 188/1000
 - 2s - loss: 1.1506e-04 - raw_multi_label_accuracy: 0.9994 - val_loss: 1.1767 - val_raw_multi_label_accuracy: 0.1639
Epoch 189/1000
 - 2s - loss: 1.1769e-04 - raw_multi_label_accuracy: 0

Epoch 244/1000
 - 2s - loss: 1.1075e-04 - raw_multi_label_accuracy: 0.9993 - val_loss: 1.2137 - val_raw_multi_label_accuracy: 0.1737
Epoch 245/1000
 - 3s - loss: 1.0788e-04 - raw_multi_label_accuracy: 0.9995 - val_loss: 1.2276 - val_raw_multi_label_accuracy: 0.1597
Epoch 246/1000
 - 2s - loss: 1.0597e-04 - raw_multi_label_accuracy: 0.9995 - val_loss: 1.2379 - val_raw_multi_label_accuracy: 0.1587
Epoch 247/1000
 - 2s - loss: 1.0824e-04 - raw_multi_label_accuracy: 0.9995 - val_loss: 1.2274 - val_raw_multi_label_accuracy: 0.1659
Epoch 248/1000
 - 2s - loss: 1.1469e-04 - raw_multi_label_accuracy: 0.9989 - val_loss: 1.2426 - val_raw_multi_label_accuracy: 0.1558
Epoch 249/1000
 - 2s - loss: 1.0437e-04 - raw_multi_label_accuracy: 0.9995 - val_loss: 1.2271 - val_raw_multi_label_accuracy: 0.1657
Epoch 250/1000
 - 3s - loss: 1.0939e-04 - raw_multi_label_accuracy: 0.9988 - val_loss: 1.2399 - val_raw_multi_label_accuracy: 0.1569
Epoch 251/1000
 - 2s - loss: 1.0671e-04 - raw_multi_label_accuracy: 0

In [25]:
def nn_output_to_predictions(res):
    label_predictions = []
    for i in range(res.shape[0]):
        pred = [0]*20
        for j in range(res.shape[1]):
            if res[i][j] >= .5:
                pred[j] = 1
        label_predictions.append(pred)
    return np.array(label_predictions)

In [26]:
predictions = nn_output_to_predictions(model_cnn.predict(x_test_seq))

In [27]:
y_test[:,2].sum()

319

In [28]:
predictions[:,2].sum()

216

In [29]:
multi_label_accuracy(y_test, predictions)

0.2220028715003589

In [30]:
multi_label_precision(y_test, predictions)

0.44844660194174774

In [31]:
multi_label_recall(y_test, predictions)

0.2714445242083432

In [32]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 85.94639865996649


In [33]:
get_per_label_metrics(y_test, predictions)

Accruacy for TV Movie: 1.0
Precision for TV Movie: 0.0
Recall for TV Movie: 0.0

Accruacy for Crime: 0.8006700167504187
Precision for Crime: 0.22727272727272727
Recall for Crime: 0.10526315789473684

Accruacy for Drama: 0.525963149078727
Precision for Drama: 0.5833333333333334
Recall for Drama: 0.3949843260188088

Accruacy for Fantasy: 0.9195979899497487
Precision for Fantasy: 0.0
Recall for Fantasy: 0.0

Accruacy for Foreign: 0.9882747068676717
Precision for Foreign: 0.0
Recall for Foreign: 0.0

Accruacy for Comedy: 0.5845896147403685
Precision for Comedy: 0.4298245614035088
Recall for Comedy: 0.4537037037037037

Accruacy for Animation: 0.9547738693467337
Precision for Animation: 0.5
Recall for Animation: 0.07407407407407407

Accruacy for Documentary: 0.9731993299832495
Precision for Documentary: 0.0
Recall for Documentary: 0.0

Accruacy for War: 0.9631490787269682
Precision for War: 0.0
Recall for War: 0.0

Accruacy for Western: 0.983249581239531
Precision for Western: 0.0
Recall for

/home/matt/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/matt/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [34]:
y_train.shape

(2388, 20)